<div class="alert alert-block alert-success">

# MLOps

Now that you've seen the individual components of an MLOps workflow with W&B, it's time to apply everything you've learned in a complete pipeline. You'll combine data versioning, experiment tracking, and hyperparameter optimization in a single comprehensive exercise.

### Your Task

Building on what we covered in previous notebooks, implement a complete MLOps pipeline that:

1. **Data Management**
   - Filter the dataset to keep only single-track events
   - Create versioned artifacts (`train_data:v1` and `val_data:v1`) with "one-track" alias, `run.log_artifact(, aliases=[..., "one-track"])`
   - Document your data processing decisions

2. **Training Pipeline**
   - Use sample_weights argument of model.fit (ensure dataset is created appropriately, i.e. x, y, sample_weigths format)
   - Use proper experiment tracking (metrics, gradients, model checkpoints)
   - Save model versions

3. **Optimization**
   - Design and execute a sweep of your choice
   - Must include at least 3 hyperparameters to optimize
   - Analyze and document the results

**Feel free to explore W&B documentation to use more advanced features!** 

</div>


# PointNet for particle flow

## Problem

This dataset contains a Monte Carlo simulation of $\rho^{\pm} \rightarrow \pi^{\pm} + \pi^0$ decays and the corresponding detector response. Specifically, the data report the measured response of **i) tracker** and **ii) calorimeter**, along with the true pyshical quantitites that generated those measurements.

<div class="alert alert-block alert-info">
This means that we expect one track per event, with mainly two energy blobs (clusters of cells) in the calorimeter.
</div>

The final **goal** is to associate the cell signals observed in the calorimeter to the track that caused those energy deposits.

## Method

The idea is to leverage a **point cloud** data representation to combine tracker and calorimeter information so to associate cell hits to the corresponding track. We will use a [**PointNet**](https://openaccess.thecvf.com/content_cvpr_2017/papers/Qi_PointNet_Deep_Learning_CVPR_2017_paper.pdf) model that is capable of handling this type of data, framed as a **semantic segmentation** approach. More precisely, this means that:
- we represent each hit in the detector as a point in the point cloud: x, y, z coordinates + additional features ("3+"-dimensional point)
- the **learning task** will be binary classification at hit level: for each cell the model learns whether its energy comes mostly from the track (class 1) or not (class 0)

## Data structure

<div class="alert alert-block alert-info">

This dataset is organized as follows:
 - for each event, we create a **sample** (i.e. point cloud)
 - each sample contains all hits in a cone around a track of the event, called **focal track**
     - the cone includes all hits within some $\Delta R$ distance of the track
     - if an event has multiple tracks, then we have more samples per event
     - since different samples have possibly different number of hits, **we pad all point clouds to ensure they have same size** (needed since the model requires inputs of same size)

</div>

## Settings & config

This section collects all configuration variables and training/model hyperparameters. 

The idea is to put it at the top so that it is easy to find and edit.

In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt

# path settings
REPO_BASEPATH = Path().cwd().parent
DATA_PATH = REPO_BASEPATH / "pnet_data/raw/rho_small.npz"
CODE_PATH = REPO_BASEPATH / "src"
sys.path.append(str(CODE_PATH))
MODEL_CHECKPOINTS_PATH = REPO_BASEPATH / "results" / "models" / "pointnet_baseline.weights.h5"

import wandb
from data_viz import *
from model_utils import *

LABELS = ["unfocus hit", "focus hit"]

# set random seed for reproducibility
SEED = 18
set_global_seeds(SEED)

# data settings
N_TRAIN, N_VAL, N_TEST = 210, 65, 50 # roughly 0.65, 0.2, 0.15

# model settings
N_FEATURES = 3
INIT_SIZE = 8
END_SIZE = 16

# training settings
BATCH_SIZE = 16
EPOCHS = 50
INIT_LR = 0.003

2024-11-27 16:27:40.286415: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 16:27:42.224175: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Model training

We proceed with model training:

1. split the data
1. build our PointNet model using Tensorflow/Keras
1. create a dataloader to feed batches into our model
1. train
1. check results

### PointNet model 

We use a PointNet model for semantic segmentation. Here is an illustration of its structure:

![PointNet architecture](../pnet_data/images/pointnet-architecture.jpg)

We have two heads:
 - classification head (used for point cloud classification)
 - segmentation head (used for semantic segmentation)

We are going to use the **segmentation head** for our problem. The architecture settings we can experiment with are:
 - `n_features` (the number of input features): original version has only size 3 as it only takes x,y,z coordinates
 - `init_size` (number of filters of first convolutional layer): original version has 64
 - `end_size` (number of filters in segmentation head): original version has 128

### Utils for new settings

You can try to implement new features alone, but you can check out some reference implementations in case you need

In [14]:
def one_track_filter(data, labels):
    multiple_samples_event_ids = [826871, 827140, 827188, 827226, 827242, 828437]
    
    one_track_mask = ~np.any(np.isin(train_data['event_number'], multiple_samples_event_ids), axis=1)
    filtered_data = data[one_track_mask]
    filtered_labels = labels[one_track_mask]
    return filtered_data, filtered_labels

def augment(point_cloud_batch, label_cloud_batch):
    noise = tf.random.uniform(
        tf.shape(point_cloud_batch[:, :, :3]), -0.001, 0.001, dtype=tf.float64
    )

    noisy_xyz = point_cloud_batch[:, :, :3] + noise
    point_cloud_batch = tf.concat([noisy_xyz, point_cloud_batch[:, :, 3:]], axis=-1)
    
    return point_cloud_batch, label_cloud_batch

def get_coords_labels_weights(point_cloud_batch, label_cloud_batch):
    category_mask = point_cloud_batch[:,:,3]
    # assign weight=1 only to cell hits (category=1)
    weights = tf.cast(tf.equal(category_mask, 1), tf.float32)
    return point_cloud_batch[:,:,:3], label_cloud_batch, weights
    
def generate_dataset(point_clouds, label_clouds, is_training=True, bs=16, n_points=800, n_features=3, labels=["unfocus hit", "focus hit"]):
    # reformat to unstructured array and transform to list of size n_samples, each element of size n_ponints x n_features
    point_clouds = structured_to_unstructured(point_clouds).astype(np.float64)
    point_clouds = [_ for _ in point_clouds]
    
    dataset = tf.data.Dataset.from_tensor_slices((point_clouds, label_clouds))
    dataset = dataset.shuffle(bs * 100) if is_training else dataset
    load_data_with_args = partial(load_data, n_points=n_points, n_features=n_features, labels=labels)
    dataset = dataset.map(load_data_with_args, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size=bs)
    dataset = (
        dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
        if is_training
        else dataset
    )
    dataset = dataset.map(get_coords_labels_weights, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

### Filtering and versioning

First step is to filter multi-track events out and update the data artifacts.

In [ ]:
SPLIT_DATA_PATH = DATA_PATH.parent.parent
NEW_SPLIT_PATH = SPLIT_DATA_PATH / "one-track"
NEW_SPLIT_PATH.mkdir(exist_ok=True, parents=True)

def read_data(split, bin_cutoff=0.5, n_classes=2, split_data_path=SPLIT_DATA_PATH): 
    filepath=str(split_data_path / f"{split}_data" / DATA_PATH.name)
    data = np.load(filepath)['feats']
    target_class = [(energy_fraction > bin_cutoff).astype(np.float32) 
                    for energy_fraction in data['truth_cell_focal_fraction_energy']]
    # target_class = (events["truth_cell_focal_fraction_energy"] > 0.5).reshape(-1)
    target_class = keras.utils.to_categorical(target_class, num_classes=n_classes)
    return data, target_class

def filter_and_version(split_data_path=NEW_SPLIT_PATH):

    with wandb.init(project="mlops-ai_infn", entity="lclissa", 
                job_type="preproc", notes="Filtering out multi-track events") as run:
        # filter data
        run.use_artifact("train_data:latest")
        run.use_artifact("val_data:latest")
        
        train_data, train_label_cloud = read_data("train", split_data_path=SPLIT_DATA_PATH)
        filtered_train_data, filtered_train_label_cloud = one_track_filter(train_data, train_label_cloud)
        # save locally
        filepath=str(split_data_path / "train_data" / DATA_PATH.name)
        np.savez(filepath, feats=filtered_train_data)
        # create new artifact tracking new version
        filtered_train_artifact = wandb.Artifact(name="train_data", type="dataset", description="One track only")
        filtered_train_artifact.add_file(local_path = str(filepath))
        run.log_artifact(filtered_train_artifact, aliases=["one-track"])
        
        train_point_clouds = train_data[input_features]
        total_training_examples = len(train_point_clouds)
        
        val_data, val_label_cloud = read_data("val", split_data_path=SPLIT_DATA_PATH)
        filtered_val_data, filtered_val_label_cloud = one_track_filter(val_data, val_label_cloud)
        # save locally
        filepath=str(split_data_path / "val_data" / DATA_PATH.name)
        np.savez(filepath, feats=filtered_val_data)
        # create new artifact tracking new version
        filtered_val_artifact = wandb.Artifact(name="val_data", type="dataset", description="One track only")
        filtered_val_artifact.add_file(local_path = str(filepath))
        run.log_artifact(filtered_val_artifact, aliases=["one-track"])

print(f"Creating new data version at: {NEW_SPLIT_PATH}")
filter_and_version(NEW_SPLIT_PATH)

## Training

Now you need to implement the training function to be passed to the sweep. A basic structure should be the following:

In [17]:
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
    
def train_wrapper():
    run = wandb.init(project="mlops-ai_infn", entity="lclissa", # name="first-sweep", # not needed as the sweep takes care of this
                job_type="sweep", notes="Playing with sweeps ...")
    cfg = run.config
    _ = run.use_artifact("train_data:latest")
    _ = run.use_artifact("val_data:latest")
    
    input_features = ["normalized_x", "normalized_y", "normalized_z", "category"]

    train_data, train_label_cloud = read_data("train", split_data_path=NEW_SPLIT_PATH)
    
    train_point_clouds = train_data[input_features]
    total_training_examples = len(train_point_clouds)
    
    val_data, val_label_cloud = read_data("val", split_data_path=NEW_SPLIT_PATH)
    val_point_clouds = val_data[input_features]
    
    print("Num train point clouds:", len(train_point_clouds))
    print("Num train point cloud labels:", len(train_label_cloud))
    print("Num val point clouds:", len(val_point_clouds))
    print("Num val point cloud labels:", len(val_label_cloud))
    
    n_points = train_point_clouds[0].shape[0]
    n_features = len(train_point_clouds[0].dtype.names)
    n_classes = len(LABELS)

    
    train_dataset = generate_dataset(train_point_clouds, train_label_cloud, 
                                 bs=cfg.batch_size, n_points=n_points, n_features=n_features, labels=LABELS)
    val_dataset = generate_dataset(val_point_clouds, val_label_cloud, is_training=False, 
                                   bs=cfg.batch_size, n_points=n_points, n_features=n_features, labels=LABELS)
    
    steps_per_epoch = total_training_examples // cfg.batch_size
    total_training_steps = steps_per_epoch * EPOCHS
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=cfg.init_lr,
        decay_steps=steps_per_epoch * 5,
        decay_rate=0.5,
        staircase=True,
    )

    segmentation_model = get_shape_segmentation_model(n_points, n_classes, n_features-1,
                                                      INIT_SIZE, END_SIZE)
    segmentation_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
        jit_compile=False
    )

    MODEL_CHECKPOINTS_PATH.parent.mkdir(exist_ok=True, parents=True)
    history = segmentation_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[
            WandbMetricsLogger(log_freq=5),
            WandbModelCheckpoint(
                       MODEL_CHECKPOINTS_PATH, #.parent / "model-{epoch:02d}-{val_loss:.2f}.weights.h5",
                       monitor="val_loss",
                       save_best_only=True,
                       save_weights_only=True,
                   )
        ],
    )
    

In [18]:
# 2: Define the search space
sweep_configuration = {
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "epoch/val_loss"},
    "parameters": {
        "init_lr": {'min': 1e-4, 'max': 1e-2,
                    'distribution': 'log_uniform' },
        "batch_size": {"values": [16, 32]},
        "init_size": {"value": [64]},
    },
}

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project="mlops-ai_infn", entity="lclissa")

wandb.agent(sweep_id, function=train_wrapper, count=2)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. init_lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: w01p3u72
Sweep URL: https://wandb.ai/lclissa/mlops-ai_infn/sweeps/w01p3u72


wandb: Agent Starting Run: okhgi8e4 with config:
wandb: 	batch_size: 16
wandb: 	init_lr: 1.001890095488338
wandb: 	init_size: 8


wandb: 🚀 View run upbeat-sweep-2 at: https://wandb.ai/lclissa/mlops-ai_infn/runs/259zu2t7
wandb: Find logs at: wandb/run-20241127_160235-259zu2t7/logs


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65
<class 'tensorflow.python.framework.ops.SymbolicTensor'>
<class 'tensorflow.python.framework.ops.SymbolicTensor'>
Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 104ms/step - accuracy: 0.5577 - loss: 164543168.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.2237 - loss: 280867328.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.7316 - loss: 16498151.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.3094 - loss: 4951713.5000 - val_accuracy: 0.1266 - val_loss: 71749880627424969607863631618244608.0000
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.7420 - loss: 1994790.1250 - val_accuracy: 0.8734 - val_loss: 23365576642744365334850701361152.0000
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 

batch/accuracy,▇▆▂▂▂█▅▁▁▄▁█▇▁▆▁▁██▁▁▁▇▇▁▁██▂▁▇████▂▁▇██
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
batch/learning_rate,██████████▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▁▂█▅▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▆▃▅▅▅▅▆▆▁█▁▆▁▇▁▇█▂▇█
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,████▄▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁
epoch/loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.88001


wandb: Agent Starting Run: xkoq2h98 with config:
wandb: 	batch_size: 16
wandb: 	init_lr: 1.0027811617519489
wandb: 	init_size: 8
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65
<class 'tensorflow.python.framework.ops.SymbolicTensor'>
<class 'tensorflow.python.framework.ops.SymbolicTensor'>
Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 99ms/step - accuracy: 0.5139 - loss: 130938392.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.3112 - loss: 890865728.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.4931 - loss: 32701370.0000 - val_accuracy: 0.8734 - val_loss: inf
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.2740 - loss: 4065840.2500 - val_accuracy: 0.8734 - val_loss: 34550737246840217424350098227200.0000
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.3032 - loss: 1623328.6250 - val_accuracy: 0.8734 - val_loss: 822558986351711500623675392.0000
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step

batch/accuracy,▃▅▆▁▁▇▄▃▁▁█▂▂██▂▂█▂▂▁▃▇██▂▁▃▅█▂███▆▅▁▃█▆
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
batch/learning_rate,██████████▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
batch/loss,▁▂▂█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▅▄▂▄▁▇▂▂▄▆█▁▆▁█▃▆▄▆▅
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,████▄▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁
epoch/loss,▅█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁█████████████████▂
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.66784
